# Trabajo de Integración Curricular

## Quick intro to code and functions

Description of **binary_labels**: a function that returns the binary representation of a range of entire numbers from 0 to $2^{num\_qubits}$.

Description of **heatmap**: it plots a matrix in a heatmap-like plot.

Description of **QuantumIsingHamiltonian**: it generates the hamiltonian of a quantum ising chain:
\begin{equation}
    H = -J\sum_{i=1}^{L-1}\sigma_{i}^{x}\sigma_{i+1}^{x} - h\sum_{i=1}^{L}\sigma_{i}^{z} \pm J\sigma_{L}^{x}\sigma_{1}^{x} 
\end{equation}
which, using the Jordan Wigner Transformation results in:
\begin{equation}
    H_{p=0,1} = -J\sum_{i=1}^{L-1}\left( c_{i}^{\dagger}c_{i+1} + c_{i}^{\dagger}c_{i+1}^{\dagger} + H.c. \right) 
    -h\sum_{i=1}^{L}\left( 1 - 2c_{i}^{\dagger}c_{i} \right)
    + (-1)^{p} J\left( c_{L}^{\dagger}c_{1} + c_{L}^{\dagger}c_{1}^{\dagger} + H.c. \right)
\end{equation}
where $c_{i},c_{i}^{\dagger}$ are the fermionic ladder operators on the i-site of the chain. I've wrote the boundary term alone, since the hamiltonian in the fermionic space does not conserve the number of fermions $N$, but it's parity $(-1)^{N}$ is conserved. That term sets the boundary conditions of the Ising chain, as follows:
- **OBC:** open boundary conditions, the term vanishes.
- **ABC:** anti-periodic boundary conditions, $p=0\rightarrow c_{L+1}=-c_{1}$.
- **PBC:** periodic boundary conditions, $p=1\rightarrow c_{L+1}=c_{1}$.
Notice that the term can be included in the spin-spin interaction operators sum if it satisfies PBC.

The global ground state is the corresponding to ABC, $p=0$, while the PBC ground state must contain an odd number of particles. The $H_{p=1}$ produces a fermion in the PBC ground state, which implies that it's energy is not the lowest, it has to be added $2J$.


**QuantumIsingHamiltonian** takes as input:
- num_qubits (int): number of qubits, equivalent to $L-1$ chain sites,
- J (float): spin-spin interaction intensity,
- h (float): transverse fiel intensity,
- periodic (bool): OBC or closed Ising Chain.
- BoundaryConditions (bool $\rightarrow$\{0,1\}): the value of p, $ABC\rightarrow p=0$, $PBC\rightarrow p=1$.
- printModel (bool): print the hamiltonian and number of qubits/chain sites.

It returns a FermionOperator and the Quadratic Hamiltonian.

## Import Libraries and Defining Functions

In [ ]:
import numpy as np
import cirq
from cirq.work.observable_measurement import measure_observables, RepetitionsStoppingCriteria
from scipy.sparse import save_npz, load_npz
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['text.color'] = 'black'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['xtick.color'] = 'black'
plt.rcParams['ytick.color'] = 'black'
plt.rcParams['legend.labelcolor'] = 'black'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
from openfermion import FermionOperator,BosonOperator,QubitOperator,bogoliubov_transform,get_sparse_operator,expectation,is_hermitian
from openfermion.utils import count_qubits,hermitian_conjugated
from openfermion.transforms import jordan_wigner,reverse_jordan_wigner,get_quadratic_hamiltonian,get_boson_operator,get_quad_operator
from openfermion.linalg import eigenspectrum,boson_ladder_sparse,boson_operator_sparse

In [ ]:
def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-45, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
def QuantumIsingHamiltonian(num_qubits,
                            J,
                            h,
                            periodic = True,
                            #SelectModes = [],
                            BoundaryConditions = 0,
                            printModel = False):

    #Initialize Fermion Model
    FermionModel = FermionOperator()
    
    #Add fermion ladder operators on correlated fermions
    for i in range(num_qubits - 1):
        FermionModel += FermionOperator(f'{i}^ {i+1}',-J)
        FermionModel += FermionOperator(f'{i}^ {i+1}^',-J)
    
    #Add hermitian conjugate
    FermionModel += hermitian_conjugated(FermionModel)
    
    #Add h field interaction
    for i in range(num_qubits):
        FermionModel += FermionOperator('',-h)
        FermionModel += FermionOperator(f'{i}^ {i}',2*h)
    
    #Add periodic term
    if periodic:
        #parity
        NParity = (-1)**(BoundaryConditions)
        PeriodicFermionModel = FermionOperator()
        PeriodicFermionModel += FermionOperator(f'{num_qubits-1}^ 0',J*NParity)
        PeriodicFermionModel += FermionOperator(f'{num_qubits-1}^ 0^',J*NParity)
        PeriodicFermionModel += hermitian_conjugated(PeriodicFermionModel)
        FermionModel += PeriodicFermionModel
    
    if printModel:
        print("Hamiltonian Function :\n",jordan_wigner(FermionModel))
        print("\nNumber of qubits : ",count_qubits(FermionModel))

    #get a Hamiltonian quadratic function from the Fermionic Operator
    Quad_Hamiltonian = get_quadratic_hamiltonian(FermionModel)

    return FermionModel, Quad_Hamiltonian

In [ ]:
def SimulateHamiltonian(num_qubits,
                        Quad_Hamiltonian,
                        InitialState = [],
                        repetitions = 10000,
                        printInitialModes = False,
                        printFullCircuit = False):
    
    #Bogoliubov Transform Matrix
    _, basis_change_matrix, _ = Quad_Hamiltonian.diagonalizing_bogoliubov_transform()
    
    #initialize qubits and circuit
    qubits = cirq.LineQubit.range(num_qubits)
    circuit = cirq.Circuit()
    
    #apply X gates to initialize state
    for qubit in range(num_qubits):
        if qubit in InitialState:
            circuit.append(cirq.X(qubits[qubit]))
        else:
            circuit.append(cirq.I(qubits[qubit]))

    #print qubit's initial state
    if printInitialModes:
        print(circuit.to_text_diagram(transpose=True))
    
    #apply Bogoliubov Transform
    circuit.append(bogoliubov_transform(qubits, basis_change_matrix),strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

    #drop negligible operations
    circuit = cirq.drop_negligible_operations(circuit)
    
    #get Final State Vector
    Final_st_vector = cirq.final_state_vector(circuit)

    #get Final Density Operator
    Density_Op = cirq.final_density_matrix(circuit)
    
    #apply measure gates
    circuit.append(cirq.measure(*qubits,key='Measures'))
    
    #print vertical circuit
    if printFullCircuit:
        print(circuit.to_text_diagram(transpose=True))
    
    #simulate circuit
    result = cirq.Simulator().run(circuit,repetitions=repetitions)

    #store as dictionary
    histogram = result.histogram(key = 'Measures')
    HistogramKeys = list(histogram.keys())
    HistogramKeys.sort()

    #get dictionary only with non-zero counts
    SortedHistogram = {bin(i)[2::].zfill(num_qubits): histogram[i]/repetitions for i in HistogramKeys}

    #get dictionary with all possible outputs
    FullHistogram = {}
    for i in range(2**num_qubits):
        if i in HistogramKeys:
            FullHistogram[bin(i)[2::].zfill(num_qubits)] = histogram[i]
        else:
            FullHistogram[bin(i)[2::].zfill(num_qubits)] = 0

    return Density_Op, Final_st_vector, FullHistogram, SortedHistogram

In [ ]:
def SimulateHamiltonianDepolarized(num_qubits,
                        Quad_Hamiltonian,
                        InitialState = [],
                        repetitions = 10000,
                        px = 0.5,
                        pz = 0.5,
                        printInitialModes = False,
                        printFullCircuit = False):
    
    #Bogoliubov Transform Matrix
    _, basis_change_matrix, _ = Quad_Hamiltonian.diagonalizing_bogoliubov_transform()
    
    #initialize qubits and circuit
    qubits = cirq.LineQubit.range(num_qubits)
    circuit = cirq.Circuit()
    
    #apply X gates to initialize state
    for qubit in range(num_qubits):
        if qubit in InitialState:
            circuit.append(cirq.X(qubits[qubit]))
        else:
            circuit.append(cirq.I(qubits[qubit]))

    #print qubit's initial state
    if printInitialModes:
        print(circuit.to_text_diagram(transpose=True))
    
    #apply Bogoliubov Transform
    circuit.append(bogoliubov_transform(qubits, basis_change_matrix),strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

    #add asymmetrical depolarized noise
    circuit.append(cirq.asymmetric_depolarize(p_x = px, p_y = 0, p_z = pz).on_each(*qubits))
    
    #get Final State Vector
    #Final_st_vector = cirq.final_state_vector(circuit)

    #get Final Density Operator
    Density_Op = cirq.final_density_matrix(circuit)
    
    #apply measure gates
    circuit.append(cirq.measure(*qubits,key='Measures'))
    
    #print vertical circuit
    if printFullCircuit:
        print(circuit.to_text_diagram(transpose=True))
    
    #simulate circuit
    result = cirq.Simulator().run(circuit,repetitions=repetitions)

    #store as dictionary
    histogram = result.histogram(key = 'Measures')
    HistogramKeys = list(histogram.keys())
    HistogramKeys.sort()

    #get dictionary only with non-zero counts
    SortedHistogram = {bin(i)[2::].zfill(num_qubits): histogram[i]/repetitions for i in HistogramKeys}

    #get dictionary with all possible outputs
    FullHistogram = {}
    for i in range(2**num_qubits):
        if i in HistogramKeys:
            FullHistogram[bin(i)[2::].zfill(num_qubits)] = histogram[i]
        else:
            FullHistogram[bin(i)[2::].zfill(num_qubits)] = 0

    return Density_Op, FullHistogram, SortedHistogram

In [ ]:
def GetEnergies(Quad_Hamiltonian, Initial_st_vector=[], Final_st_vector=[], printEnergies = False):
    orbital_energies, constant = Quad_Hamiltonian.orbital_energies()
    
    # Compute the expectation value of the final state with the Hamiltonian
    Quad_Hamiltonian_sparse = get_sparse_operator(Quad_Hamiltonian)

    if printEnergies:
        print(f"Orbital energies : {orbital_energies}")
        print(f"Constant : {constant}")
        # Print out the ground state energy; it should match
        print(f"Ground Energy : {Quad_Hamiltonian.ground_energy()}")

        try:
            print(f"\nExpected Energy with Initial State : {expectation(Quad_Hamiltonian_sparse, Initial_st_vector)}")
        except:
            print("\nNon-Valid/None Initial State provided")
    
        try:
            print(f"Expected Energy with Final State : {expectation(Quad_Hamiltonian_sparse, Final_st_vector)}")
        except:
            print("Non-Valid/None Final State provided")

    return np.array(orbital_energies),Quad_Hamiltonian.ground_energy()

In [ ]:
def ExpectedMagnetization(num_qubits,results):
    avg_sigma = 0

    #count the N - 2*(number of ones in each output)
    for i in results.keys():
        avg_sigma += (num_qubits - 2*(i.count('1')))*results[i]
    
    return avg_sigma/num_qubits

In [ ]:
def Correlation(num_qubits, Quad_Hamiltonian, qi, qj, repetitions, InitialState = []):
    #Bogoliubov Transform Matrix
    _, basis_change_matrix, _ = Quad_Hamiltonian.diagonalizing_bogoliubov_transform()
    #initialize qubits and circuit
    qubits = cirq.LineQubit.range(num_qubits)
    circuit = cirq.Circuit()
    #apply X gates to initialize state
    for qubit in range(num_qubits):
        if qubit in InitialState:
            circuit.append(cirq.X(qubits[qubit]))
        else:
            circuit.append(cirq.I(qubits[qubit]))
    #apply Bogoliubov Transform
    circuit.append(bogoliubov_transform(qubits, basis_change_matrix),strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)
    #define gates
    Pauli_gates = {"Z" : cirq.Z, "X" : cirq.X}
    #define the Pauli Observables
    observables = [Pauli_gates[qi].on(qubits[0])*Pauli_gates[qj].on(qubits[i]) for i in range(1,num_qubits)]
    #simulate the observables
    result = measure_observables(circuit, observables, cirq.Simulator(), stopping_criteria = RepetitionsStoppingCriteria(repetitions))
    #output correlation
    correlation = [i.mean for i in result]
    return correlation

In [ ]:
def SimulateEnergy(num_qubits, J, h, periodic = True, BoundaryConditions = 0, InitialState = [],repetitions=10000):
    _,IsingH = QuantumIsingHamiltonian(num_qubits, J, h, periodic, BoundaryConditions)
    
    #Bogoliubov Transform Matrix
    _, basis_change_matrix, _ = IsingH.diagonalizing_bogoliubov_transform()
    #initialize qubits and circuit
    qubits = cirq.NamedQubit.range(num_qubits,prefix='q')
    circuit = cirq.Circuit()

    #apply X gates to initialize state
    for qubit in range(num_qubits):
        if qubit in InitialState:
            circuit.append(cirq.X(qubits[qubit]))
        else:
            circuit.append(cirq.I(qubits[qubit]))
    
    #apply Bogoliubov Transform
    circuit.append(bogoliubov_transform(qubits, basis_change_matrix),strategy=cirq.circuits.InsertStrategy.NEW_THEN_INLINE)

    #define Pauli Operator
    interspins = [cirq.PauliString(-J,cirq.X(qubits[i]),cirq.X(qubits[i+1])) for i in range(num_qubits-1)]
    singlespins = [cirq.PauliString(-h,cirq.Z(qubits[i])) for i in range(num_qubits)]
    H = interspins + singlespins
    if periodic:
        periodic_term = [(-1)**BoundaryConditions,cirq.Y(qubits[0])]
        periodic_term += [cirq.Z(qubits[i]) for i in range(1,num_qubits-1)]
        periodic_term += [cirq.Y(qubits[-1])]
        H += [cirq.PauliString(periodic_term)]
    HOp = cirq.PauliSum.from_pauli_strings(H)

    #simulate the observables
    collector = cirq.PauliSumCollector(circuit=circuit, observable=HOp, samples_per_term = repetitions)
    collector.collect(sampler=cirq.Simulator())
    energy = collector.estimated_energy()

    return energy

## Transverse Field Ising Model

In [ ]:
#Set parameters
num_qubits = 2 #number of qubits to simulate
J = 1. #spin-spin interaction intensity 
h = 0. #transverse field intensitiy

In [ ]:
FermionIsing,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                  J = J,
                                  h = h,
                                  periodic = True,
                                  BoundaryConditions = 1,
                                  printModel = True);
Density_Operator, Ground_State, FullHistogram, SortedHistogram = SimulateHamiltonian(num_qubits = num_qubits,
                                                                      Quad_Hamiltonian = IsingH,
                                                                      #InitialState=[0,1],
                                                                      InitialState=[0],
                                                                      printFullCircuit=False);
gap_energies, ground_energy = GetEnergies(Quad_Hamiltonian=IsingH,
                                          Final_st_vector=Ground_State,
                                          printEnergies=True);
print(f"Final State : {cirq.dirac_notation(Ground_State)}")

In [ ]:
plt.bar(FullHistogram.keys(),FullHistogram.values())
#plt.title("Counts vs State (All States)")
plt.xlabel("State")
plt.ylabel("Counts")
plt.xticks(rotation = 90)
plt.savefig("Measurements_4_spins_1_05_pbc.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

print(f"Number of total outcomes : {len(FullHistogram)}")

In [ ]:
plt.bar(SortedHistogram.keys(),SortedHistogram.values(), color = 'tab:blue')
plt.title("Probability vs State (Non Zero States)")
plt.xlabel("State")
plt.ylabel("Probability")
plt.xticks(rotation = 90)
plt.show()

print(f"Number of possible outcomes : {len(SortedHistogram)}")

In [ ]:
fig, (ax,ax2) = plt.subplots(1,2,figsize=plt.figaspect(0.4))
ticks_for_map = binary_labels(num_qubits)

im, _ = heatmap(np.real(Density_Operator), ticks_for_map, ticks_for_map, ax=ax,
                   cmap="viridis", cbarlabel="Real")
#texts = annotate_heatmap(im, valfmt="{x:.1f} t")

im, _ = heatmap(np.imag(Density_Operator), ticks_for_map, ticks_for_map, ax=ax2,
                   cmap="inferno", cbarlabel="Imaginary")

fig.tight_layout()
plt.show()

In [ ]:
_ = cirq.plot_density_matrix(Density_Operator)

## States to see Parity Breaking

In [ ]:
repetitions = 10000
num_qubits = 4
J = 1.

h_values = np.linspace(0,2,9)

results_abc = []
results_pbc = []

for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                          J = J,
                                          h = h_values[i],
                                          periodic = True,
                                          BoundaryConditions = 1)
    _,_,HistogramResults,_ = SimulateHamiltonian(num_qubits = num_qubits,
                                                Quad_Hamiltonian = IsingH,
                                                InitialState = [],
                                                repetitions = repetitions); 
    results_pbc.append(list(HistogramResults.values()))

    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                          J = J,
                                          h = h_values[i],
                                          periodic = True,
                                          BoundaryConditions = 0)
    _,_,HistogramResults,_ = SimulateHamiltonian(num_qubits = num_qubits,
                                                Quad_Hamiltonian = IsingH,
                                                InitialState = [],
                                                repetitions = repetitions); 
    results_abc.append(list(HistogramResults.values()))

In [ ]:
fig = plt.figure(figsize = (14,8))
ax1 = fig.add_subplot(121,projection='3d')
ax2 = fig.add_subplot(122,projection='3d')

colors_blue = plt.colormaps['winter'](np.linspace(1,0,len(h_values)))
colors_red = plt.colormaps['autumn'](np.linspace(1,0,len(h_values)))
colors_jet = plt.colormaps['jet'](np.linspace(0,1,len(h_values)))

states = binary_labels(num_qubits)

#yticks = [3, 2, 1, 0]
for i, r in enumerate(results_abc):
    ax1.bar(states, list(map(lambda x: x/repetitions, r)), zs = h_values[i], zdir='y', color=colors_red[i], alpha=0.9)
for i, r in enumerate(results_pbc):
    ax2.bar(states, list(map(lambda x: x/repetitions, r)), zs = h_values[i], zdir='y', color=colors_blue[i], alpha=0.9)

ax1.set_xlabel('Computational states')
ax1.xaxis.labelpad = 4
ax1.tick_params(axis='x', pad=-5)
ax1.set_xticklabels(states, rotation = 90)
ax1.set_ylabel('h')
ax1.set_ylim([0,2])
ax1.tick_params(axis='y', pad=-2)
ax1.set_yticklabels(h_values, rotation = -45)
ax1.set_title('ABC')

ax2.set_xlabel('Computational states')
ax2.xaxis.labelpad = 4
ax2.tick_params(axis='x', pad=-5)
ax2.set_xticklabels(states, rotation = 90)
ax2.set_ylabel('h')
ax2.set_ylim([0,2])
ax2.tick_params(axis='y', pad=-2)
ax2.set_yticklabels(h_values, rotation = -45)
ax2.set_title('PBC')

plt.savefig("Computational_states_abc_pbc.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Z - Magnetization

In [ ]:
num_qubits = [2]#,6,8,10]
J = 1.

h_values = np.linspace(0,2,81)

M_abc = np.empty([len(h_values),len(num_qubits)],dtype=float)
M_pbc = np.empty([len(h_values),len(num_qubits)],dtype=float)
M_obc = np.empty([len(h_values),len(num_qubits)],dtype=float)

for l in range(len(num_qubits)):
    for i in range(len(h_values)):
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          periodic = True,
                                          BoundaryConditions = 1)
                                          #SelectModes = InitialMode); 
        _,_,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                    Quad_Hamiltonian = IsingH,
                                                    InitialState = InitialState,
                                                    repetitions = 1000);
        M_pbc[i][l] = ExpectedMagnetization(num_qubits = num_qubits[l],
                                     results = HistogramResults)
    
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          periodic = True,
                                          BoundaryConditions = 0)
                                          #SelectModes = InitialMode);
        _,_,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                    Quad_Hamiltonian = IsingH,
                                                    InitialState = [],
                                                    repetitions = 1000);
        M_abc[i][l] = ExpectedMagnetization(num_qubits = num_qubits[l],
                                     results = HistogramResults)

        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          periodic = False)
                                          #SelectModes = InitialMode);
        _,_,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                    Quad_Hamiltonian = IsingH,
                                                    InitialState = [],
                                                    repetitions = 1000);
        M_obc[i][l] = ExpectedMagnetization(num_qubits = num_qubits[l],
                                     results = HistogramResults)

In [ ]:
def exact(lam):
    if lam < -1:
        return -1/2+lam/(2*np.sqrt(1+lam**2))
    if lam < 1:
        return lam/(2*np.sqrt(1+lam**2))
    if lam > 1:
        return 1/2+lam/(2*np.sqrt(1+lam**2))

fig, ax = plt.subplots(figsize = (8,4))

#ax.axhline(y = 0, color = 'k', ls = '--')
#ax.axvline(x = -1, color = 'k', ls = '--')
ax.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax.plot(h_values,M_abc[:,0],'g.',label = 'ABC')
ax.plot(h_values,M_obc[:,0],'k.',label = 'OBC')
ax.plot(h_values,list(map(lambda x: exact(x),h_values)),color = 'r', ls = '-' ,label = 'Exact solution\nfor PBC')
ax.plot(h_values,M_pbc[:,0],'b.',label = 'PBC')
ax.set_xlabel("h")
ax.set_ylabel("$\\langle\sigma^{z}\\rangle$ of the Ground State")
#ax.set_title("Magnetization")
ax.legend(loc = 'lower right', frameon = False, ncols = 2)
#plt.savefig("Magnetization_4_spins.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=plt.figaspect(0.9))

colors_blue = plt.colormaps['winter'](np.linspace(1,0,len(num_qubits)))
colors_red = plt.colormaps['autumn'](np.linspace(1,0,len(num_qubits)))

for i,j in enumerate(num_qubits):
    ax1.plot(h_values,M_abc[:,i],marker = '',ls = '-', color = colors_blue[i], label = f'{j} spins')
    ax2.plot(h_values,M_pbc[:,i],marker = '',ls = '-', color = colors_red[i], label = f'{j} spins')

ax1.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax2.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax2.set_xlabel('h')
ax1.set_ylabel('$\\langle\sigma^{z}\\rangle$ for ABC')
ax2.set_ylabel('$\\langle\sigma^{z}\\rangle$ for PBC')
ax1.legend(bbox_to_anchor=(1.3,0.5), loc = 'right', frameon = False)
ax2.legend(bbox_to_anchor=(1.3,0.5), loc = 'right', frameon = False)
ax1.set_ylim(-0.1,1.1)
ax2.set_ylim(-0.1,1.1)
ax1.set_xticks(np.linspace(0,2,9))
ax2.set_xticks(np.linspace(0,2,9))

plt.savefig("Magnetization_2_2_10.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

In [ ]:
num_qubits = [2,4,6,8,10]

h = 1.
J_values = np.linspace(0,2,81)

M_abc = np.empty([len(J_values),len(num_qubits)],dtype=float)
M_pbc = np.empty([len(J_values),len(num_qubits)],dtype=float)

for l in range(len(num_qubits)):
    for i in range(len(J_values)):
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J_values[i],
                                          h = h,
                                          periodic = True,
                                          BoundaryConditions = 1)
                                          #SelectModes = InitialMode);
        
        _,_,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                    Quad_Hamiltonian = IsingH,
                                                    #SelectModes = [],
                                                    repetitions = 3000);
    
        M_pbc[i,l] = ExpectedMagnetization(num_qubits = num_qubits[l],
                                     results = HistogramResults)
    
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J_values[i],
                                          h = h,
                                          periodic = True,
                                          BoundaryConditions = 0)
                                          #SelectModes = InitialMode);
        
        _,_,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                    Quad_Hamiltonian = IsingH,
                                                    #SelectModes = [],
                                                    repetitions = 3000);
    
        M_abc[i,l] = ExpectedMagnetization(num_qubits = num_qubits[l],
                                     results = HistogramResults)

In [ ]:
fig, ax = plt.subplots()

ax.plot(J_values,np.zeros(len(J_values)),'k--')
#ax.axvline(x = -1, color = 'k', ls = '--')
ax.axvline(x = 1, color = 'k', ls = '--')
ax.plot(J_values,M_ground,'b.',label = 'Ground State (P=0)')
ax.plot(J_values,M_1ext,'r.',label = 'Ground State (P=1)')
ax.set_xlabel("J")
ax.set_ylabel("$< \sigma_z >$")
ax.set_title("Magnetization")
ax.legend(loc = 'upper right')
plt.show()

In [ ]:
fig, (ax1,ax2) = plt.subplots(2,1,figsize=plt.figaspect(0.9))

colors_blue = plt.colormaps['winter'](np.linspace(1,0,len(num_qubits)))
colors_red = plt.colormaps['autumn'](np.linspace(1,0,len(num_qubits)))

for i,j in enumerate(num_qubits):
    ax1.plot(J_values,M_abc[:,i],marker = '',ls = '-', color = colors_blue[i], label = f'{j} spins')
    ax2.plot(J_values,M_pbc[:,i],marker = '',ls = '-', color = colors_red[i], label = f'{j} spins')

ax1.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax2.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax2.set_xlabel('J')
ax1.set_ylabel('$\\langle\sigma^{z}\\rangle$ for ABC')
ax2.set_ylabel('$\\langle\sigma^{z}\\rangle$ for PBC')
ax1.legend(bbox_to_anchor=(1.3,0.5), loc = 'right', frameon = False)
ax2.legend(bbox_to_anchor=(1.3,0.5), loc = 'right', frameon = False)
ax1.set_ylim(-0.1,1.1)
ax2.set_ylim(-0.1,1.1)
ax1.set_xticks(np.linspace(0,2,9))
ax2.set_xticks(np.linspace(0,2,9))

plt.savefig("Magnetization_2_2_10_vs_J.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Z - Magnetization with noise

In [ ]:
num_qubits = 4
J = 1.

h_values = np.linspace(0,3,100)

M_ground = np.empty(len(h_values),dtype=float)
M_1ext = np.empty(len(h_values),dtype=float)

#InitialMode = []
for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                      J = J,
                                      h = h_values[i],
                                      periodic = True,
                                      BoundaryConditions = 0)
                                      #SelectModes = InitialMode);
    
    _,_,HistogramResults = SimulateHamiltonianDepolarized(num_qubits = num_qubits,
                                                Quad_Hamiltonian = IsingH,
                                                #SelectModes = InitialMode,
                                                repetitions = 1000,
                                                px = 0.1,
                                                pz = 0.1);

    M_ground[i] = ExpectedMagnetization(num_qubits = num_qubits,
                                 results = HistogramResults)

#InitialMode = [0]
for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                      J = J,
                                      h = h_values[i],
                                      periodic = True,
                                      BoundaryConditions = 1)
                                      #SelectModes = InitialMode);
    
    _,_,HistogramResults = SimulateHamiltonianDepolarized(num_qubits = num_qubits,
                                                Quad_Hamiltonian = IsingH,
                                                #SelectModes = [], #simulate ground state
                                                repetitions = 1000,
                                                px = 0.1,
                                                pz = 0.1);

    M_1ext[i] = ExpectedMagnetization(num_qubits = num_qubits,
                                 results = HistogramResults)

In [ ]:
def exact(lam):
    if lam < -1:
        return -1/2+lam/(2*np.sqrt(1+lam**2))
    if lam < 1:
        return lam/(2*np.sqrt(1+lam**2))
    if lam > 1:
        return 1/2+lam/(2*np.sqrt(1+lam**2))

fig, ax = plt.subplots()

ax.axhline(y = 0, color = 'k', ls = '--')
#ax.axvline(x = -1, color = 'k', ls = '--')
ax.axvline(x = 1, color = 'k', ls = '--')
ax.plot(h_values,list(map(lambda x: exact(x),h_values)),'g-',label = 'Exact Solution')
ax.plot(h_values,M_ground,'b.',label = 'Ground State (P=0)')
ax.plot(h_values,M_1ext,'r.',label = 'Ground State (P=1)')
ax.set_xlabel("h")
ax.set_ylabel("$< \sigma_z >$")
ax.set_title("Magnetization")
ax.legend(loc = 'upper left')
plt.show()

At higher values of px, magnetization tends to 0. The behavior at the critical point is kept.
At higher values of pz, magnetization is invariant.

## Ground energy and energy gaps

In [ ]:
num_qubits = 4
J = 1.

h_values = np.linspace(0,2,81)
E_abc = np.empty([len(h_values),num_qubits + 1],dtype=float)
E_pbc = np.empty([len(h_values),num_qubits + 1],dtype=float)
E_obc = np.empty([len(h_values),num_qubits + 1],dtype=float)

for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                      J = J,
                                      h = h_values[i],
                                      periodic = True,
                                      BoundaryConditions = 1)
                                      #SelectModes = InitialMode);
    orbital_energies, ground_energy = GetEnergies(Quad_Hamiltonian = IsingH);
    
    E_pbc[i][0] = ground_energy
    for j in range(1,num_qubits + 1):
        E_pbc[i][j] = orbital_energies[j-1] 

    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                      J = J,
                                      h = h_values[i],
                                      periodic = True,
                                      BoundaryConditions = 0)
                                      #SelectModes = InitialMode);
    orbital_energies, ground_energy = GetEnergies(Quad_Hamiltonian = IsingH);
    
    E_abc[i][0] = ground_energy
    for j in range(1,num_qubits + 1):
        E_abc[i][j] = orbital_energies[j-1] 
        
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                      J = J,
                                      h = h_values[i],
                                      periodic = False)
                                      #SelectModes = InitialMode);
    orbital_energies, ground_energy = GetEnergies(Quad_Hamiltonian = IsingH);
    
    E_obc[i][0] = ground_energy
    for j in range(1,num_qubits + 1):
        E_obc[i][j] = orbital_energies[j-1] 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(8,8))

lss = ['0','-','--','-.',':']
#colors = ['0', 'orangered', 'navy', 'springgreen', 'blueviolet']
colors = ['0', 'red', 'blue', 'orange', 'green']

for i in range(1,num_qubits+1):
    ax1.plot(h_values,E_pbc[:,i], ls = lss[i], lw = 2, color = colors[i], label = '$\\Delta$E' + f'{i}')
    ax2.plot(h_values,E_abc[:,i], ls = lss[i], lw = 2, color = colors[i])#, label = f'Orbital {i} (P=0)')
    ax3.plot(h_values,E_obc[:,i], ls = lss[i], lw = 2, color = colors[i])#, label = f'Orbital {i} (P=0)')

for ax in (ax1,ax2,ax3):
    ax.axhline(y = 0, color = 'k', ls = '--', lw = 0.8)
    ax.axhline(y = 2*J, color = 'k', ls = '--', lw = 0.8)
    ax.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
    ax.set_ylim(-0.5,6.5)
    ax.set_ylabel("Energy")

ax1.set_title("PBC", x = 0.01, y=1.0, pad = -14, loc = 'left')
ax2.set_title("ABC", x = 0.01, y=1.0, pad = -14, loc = 'left')
ax3.set_title("OBC", x = 0.01, y=1.0, pad = -14, loc = 'left')
ax3.set_xlabel("h")

ax1.legend(bbox_to_anchor=(0.8,1.3), loc = 'upper right',ncols = 4, frameon = False)
plt.savefig("Energy_orbitals_4_spins.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Ready to simulate Ground Energy 

In [ ]:
num_qubits = [4]#,6,8,10]
J = 1.

h_values = np.linspace(0,2,81)

E_abc = np.empty([len(h_values),len(num_qubits)],dtype=float)
E_pbc = np.empty([len(h_values),len(num_qubits)],dtype=float)
E_obc = np.empty([len(h_values),len(num_qubits)],dtype=float)
E_abc_n = np.empty([len(h_values),len(num_qubits)],dtype=float)
E_pbc_n = np.empty([len(h_values),len(num_qubits)],dtype=float)
E_obc_n = np.empty([len(h_values),len(num_qubits)],dtype=float)

for l in range(len(num_qubits)):
    for i in range(len(h_values)):
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                      J = J,
                                      h = h_values[i],
                                      periodic = True,
                                      BoundaryConditions = 1)
        _, E_pbc_n[i][l] = GetEnergies(Quad_Hamiltonian = IsingH);
        E_pbc[i][l] = SimulateEnergy(num_qubits = num_qubits[l],
                                    J = J,
                                    h = h_values[i],
                                    periodic = True,
                                    BoundaryConditions = 1,
                                    InitialState = [],
                                    repetitions = 1000)
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                      J = J,
                                      h = h_values[i],
                                      periodic = True,
                                      BoundaryConditions = 0)
        _, E_abc_n[i][l] = GetEnergies(Quad_Hamiltonian = IsingH);
        E_abc[i][l] = SimulateEnergy(num_qubits = num_qubits[l],
                                    J = J,
                                    h = h_values[i],
                                    periodic = True,
                                    BoundaryConditions = 0,
                                    InitialState = [],
                                    repetitions = 1000)
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                      J = J,
                                      h = h_values[i],
                                      periodic = False)
        _, E_obc_n[i][l] = GetEnergies(Quad_Hamiltonian = IsingH);
        E_obc[i][l] = SimulateEnergy(num_qubits = num_qubits[l],
                                    J = J,
                                    h = h_values[i],
                                    periodic = False,
                                    InitialState = [],
                                    repetitions = 1000)

In [ ]:
fig, ax = plt.subplots(figsize = (8,4))

ax.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax.plot(h_values,E_obc_n[:,0],color = 'gray', ls = '-',label = 'OBC')
ax.plot(h_values,E_abc_n[:,0],color = 'lawngreen',ls = '-',label = 'ABC')
ax.plot(h_values,E_pbc_n[:,0],color = 'turquoise',ls = '-',label = 'PBC')
ax.plot(h_values,E_obc[:,0],'k.',label = 'OBC')
ax.plot(h_values,E_abc[:,0],'g.',label = 'ABC')
ax.plot(h_values,E_pbc[:,0],'b.',label = 'PBC')
ax.set_xlabel("h")
ax.set_ylabel("Ground state energy")
ax.legend(loc = 'upper right', frameon = False, ncols = 2)
#plt.savefig("SimulatedGroundEnergy.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## ABC - PBC Ground Energies

In [ ]:
num_qubits = 14
num_modes = range(2,num_qubits + 1)

J = 1.
h_values = [0.1] + [0.25*i for i in range(1,9)]

diff_Energies = np.empty([len(num_modes),len(h_values)],dtype=float)

for i in range(len(num_modes)):
    for j in range(len(h_values)):
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_modes[i],
                                          J = J,
                                          h = h_values[j],
                                          periodic = True,
                                          BoundaryConditions = 0)
                                          #SelectModes = []);
        _, ground_energy_ABC = GetEnergies(Quad_Hamiltonian = IsingH);

        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_modes[i],
                                          J = J,
                                          h = h_values[j],
                                          periodic = True,
                                          BoundaryConditions = 1)
                                          #SelectModes = [0]);
        _, ground_energy_PBC = GetEnergies(Quad_Hamiltonian = IsingH);

        diff_Energies[i][j] = ground_energy_PBC - ground_energy_ABC

In [ ]:
fig, ax = plt.subplots(figsize = (8,4))

colors_blue = plt.colormaps['winter'](np.linspace(1,0,len(h_values)//2))
colors_red = plt.colormaps['autumn'](np.linspace(1,0,len(h_values)//2))

for i,h in enumerate(h_values):
    if h < 1.0:
        ax.plot(num_modes,diff_Energies[:,i], color = colors_blue[i], marker = '.', label = f'h = {h}')
    elif h > 1.0:
        ax.plot(num_modes,diff_Energies[:,i], color = colors_red[i//2 - 1], marker = '.', label = f'h = {h}')
    else:
        ax.plot(num_modes,diff_Energies[:,i], color = 'black', marker = '.', label = f'h = {1.0}')
#ax.axhline(y = 0, color = 'k', ls = '--')
#ax.set_title('$E_{0}^{PBC} - E_{0}^{ABC} = \Delta E_{0}$ vs N')
ax.set_yscale("log")
ax.set_ylabel("$|E_{GS}^{PBC} - E_{GS}^{ABC}|$")
ax.set_xlabel("Number of spins")
ax.set_xticks(range(2,num_qubits + 1))
ax.legend(loc = 'lower left', ncols = 3, frameon = False)
plt.savefig("Ground_Energy_Difference_vs_Number_Spins.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

In [ ]:
num_qubits = [2,4,6,8,10,12,14]
J = 1.
h_values = np.linspace(0,3,81)

diff_Energies = np.empty([len(h_values),len(num_qubits)],dtype=float)

for l in range(len(num_qubits)):
    for i in range(len(h_values)):
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          BoundaryConditions = 0)
        _,E_ABC = GetEnergies(Quad_Hamiltonian = IsingH)
    
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          BoundaryConditions = 1)
        _,E_PBC = GetEnergies(Quad_Hamiltonian = IsingH)
    
        diff_Energies[i,l] = E_PBC - E_ABC

In [ ]:
fig, ax = plt.subplots(figsize = (8,4))

colors_blue = plt.colormaps['winter'](np.linspace(0,1,len(num_qubits)))

for i, q in enumerate(num_qubits):
    ax.plot(h_values,diff_Energies[:,i], marker = '.', ls = '-', color = colors_blue[i], label = f'{q} spins')
#ax.axhline(y = 0, color = 'k', ls = '--', lw = 0.8)
#ax.axvline(x = -1, color = 'k', ls = '--')
ax.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax.set_xlabel("h")
ax.set_ylabel("$|E_{GS}^{PBC}-E_{GS}^{ABC}|$")
#ax.set_title("Energy Diff")
ax.legend(bbox_to_anchor=(0.87,1.2), loc = 'upper right', ncols = 4, frameon = False)
plt.savefig("Ground_Energy_Difference_vs_h.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Correlation funtions of spins 

In [ ]:
num_qubits = 14
J = 1.
h_values = [0.5,0.75,1.,1.25,1.5]
corr_x_abc = np.empty([len(h_values),num_qubits - 1],dtype=float)
corr_x_pbc = np.empty([len(h_values),num_qubits - 1],dtype=float)

for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                       J = J,
                                       h = h_values[i],
                                       periodic = True,
                                       BoundaryConditions = 0);
    _,Ground_State,_,_ = SimulateHamiltonian(num_qubits = num_qubits,
                                             Quad_Hamiltonian = IsingH,
                                             repetitions = 1);
    for j in range(num_qubits-1):
        Corr_op = QubitOperator(f'X0 X{j + 1}',1.)
        corr_x_abc[i][j] = np.real(expectation(get_sparse_operator(Corr_op, n_qubits = num_qubits),Ground_State))

    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                        J = J,
                                        h = h_values[i],
                                        periodic = True,
                                        BoundaryConditions = 1);
    _,Ground_State,_,_ = SimulateHamiltonian(num_qubits = num_qubits,
                                            Quad_Hamiltonian = IsingH,
                                            repetitions = 1);
    for j in range(num_qubits-1):
        Corr_op = QubitOperator(f'X0 X{j + 1}',1.)
        corr_x_pbc[i][j] = np.real(expectation(get_sparse_operator(Corr_op, n_qubits = num_qubits),Ground_State))

In [ ]:
def H_corr_x(n):
    H = 1.
    for i in range(1,n):
        H *= i**(n-i)
    return H

def corr_x_th(num_qubits, n):
    if n <= num_qubits/2:
        corr = np.power(2/np.pi,n)*(2**(2*n*(n-1)))*(np.power(H_corr_x(n),4)/H_corr_x(2*n))
    else:
        x = num_qubits - n
        corr = np.power(2/np.pi,x)*(2**(2*x*(x-1)))*(np.power(H_corr_x(x),4)/H_corr_x(2*x))
    return corr

fig, (ax1,ax2) = plt.subplots(1,2,figsize = (12,6))

ax1.plot(range(1,num_qubits),list(map(lambda x: corr_x_th(num_qubits,x),range(1,num_qubits))),'r--',label = 'Theoretical\nat c.p.')
ax2.plot(range(1,num_qubits),list(map(lambda x: corr_x_th(num_qubits,x),range(1,num_qubits))),'r--',label = 'Theoretical\nat c.p.')

for i in range(len(h_values)):
    ax1.plot(range(1,num_qubits),corr_x_pbc[i][:], ls = ':', marker = "o", label = f"h = {h_values[i]}")
    ax2.plot(range(1,num_qubits),corr_x_abc[i][:], ls = ':', marker = "o", label = f"h = {h_values[i]}")

ax1.set_title("Correlation between 0 and i-th spin PBC")
ax1.set_ylabel("<$\sigma_{0}^{x}\sigma_{i}^{x}$>")
ax1.set_xlabel("Spin index")
ax1.legend(loc = 'upper right')
ax2.set_title("Correlation between 0 and i-th spin ABC")
ax2.set_ylabel("<$\sigma_{0}^{x}\sigma_{i}^{x}$>")
ax2.set_xlabel("Spin index")
ax2.legend(loc = 'upper right')
plt.show()

In [ ]:
num_qubits = 14
J = 1.
h_values = np.linspace(0.5,1.5,3)
corr_z_even = np.empty([len(h_values),num_qubits - 1],dtype=float)
corr_z_odd = np.empty([len(h_values),num_qubits - 1],dtype=float)

for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                       J = J,
                                       h = h_values[i],
                                       BoundaryConditions = 0);
    _,Ground_State,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits,
                                             Quad_Hamiltonian = IsingH,
                                             repetitions = 3000);
    magnetization_z = ExpectedMagnetization(num_qubits = num_qubits, results = HistogramResults)
    for j in range(num_qubits-1):
        Corr_op = QubitOperator(f'Z0 Z{j + 1}',1.)
        corr_z_even[i][j] = np.real(expectation(get_sparse_operator(Corr_op, n_qubits = num_qubits),Ground_State)) - magnetization_z*magnetization_z

    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                        J = J,
                                        h = h_values[i],
                                        BoundaryConditions = 1);
    _,Ground_State,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits,
                                            Quad_Hamiltonian = IsingH,
                                            repetitions = 3000);
    magnetization_z = ExpectedMagnetization(num_qubits = num_qubits, results = HistogramResults)
    for j in range(num_qubits-1):
        Corr_op = QubitOperator(f'Z0 Z{j + 1}',1.)
        corr_z_odd[i][j] = np.real(expectation(get_sparse_operator(Corr_op, n_qubits = num_qubits),Ground_State)) - magnetization_z*magnetization_z


In [ ]:
def corr_z_th(num_qubits, n):
    if n <= num_qubits/2:
        corr = (2/np.pi)*(2/np.pi)*(1/(4*n*n - 1))
    else:
        corr = (2/np.pi)*(2/np.pi)*(1/(4*(num_qubits - n)*(num_qubits - n) - 1))
    return corr

fig, (ax1,ax2) = plt.subplots(1,2,figsize = (12,6))

#ax1.plot(range(1,num_qubits),list(map(lambda x: corr_z_th(num_qubits,x),range(1,num_qubits))),'r--',label = 'Theoretical\nat c.p.')
ax2.plot(range(1,num_qubits),list(map(lambda x: corr_z_th(num_qubits,x),range(1,num_qubits))),'r--',label = 'Theoretical\nat c.p.')

for i in range(len(h_values)):
    ax1.plot(range(1,num_qubits),corr_z_odd[i][:],ls = ':',marker = "o",label = f"h = {h_values[i]}")
    ax2.plot(range(1,num_qubits),corr_z_even[i][:],ls = ':',marker = "o",label = f"h = {h_values[i]}")

ax1.set_title("Correlation between 0-th and i-th spin ABC")
ax1.set_ylabel("<$\sigma_{0}^{z}\sigma_{z}^{x}$> - <$\sigma_{z}$>$^{2}$")
ax1.set_xlabel("Spin index")
ax1.legend(loc = 'upper center')
ax2.set_title("Correlation between 0-th and i-th spin PBC")
ax2.set_ylabel("<$\sigma_{0}^{z}\sigma_{i}^{z}$>")
ax2.set_xlabel("Spin index")
ax2.legend(loc = 'upper center')
plt.show()

## Ready to Sim Correlation Functions

In [ ]:
num_qubits = 14
J = 1.
h_values = [(1+0.25*i) for i in range(8)]
corr_x_abc = np.empty([len(h_values),num_qubits - 1],dtype=float)
corr_x_pbc = np.empty([len(h_values),num_qubits - 1],dtype=float)

for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                       J = J,
                                       h = h_values[i],
                                       BoundaryConditions = 0);
    corr_h = Correlation(num_qubits, IsingH, "X", "X", 30000)
    for j in range(num_qubits-1):
        corr_x_abc[i][j] = corr_h[j]

    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                        J = J,
                                        h = h_values[i],
                                        BoundaryConditions = 1);
    corr_h = Correlation(num_qubits, IsingH, "X", "X", 30000)
    for j in range(num_qubits-1):
        corr_x_pbc[i][j] = corr_h[j]

In [ ]:
def H_corr_x(n):
    H = 1.
    for i in range(1,n):
        H *= i**(n-i)
    return H

def corr_x_th(num_qubits, n):
    if n <= num_qubits/2:
        corr = np.power(2/np.pi,n)*(2**(2*n*(n-1)))*(np.power(H_corr_x(n),4)/H_corr_x(2*n))
    else:
        x = num_qubits - n
        corr = np.power(2/np.pi,x)*(2**(2*x*(x-1)))*(np.power(H_corr_x(x),4)/H_corr_x(2*x))
    return corr

fig, (ax1,ax2) = plt.subplots(1,2,figsize = (8,4))

colors_blue = plt.colormaps['winter'](np.linspace(0,1,len(h_values)))

ax1.plot(range(2,num_qubits+1),list(map(lambda x: corr_x_th(num_qubits,x),range(1,num_qubits))),'r-',
         label = 'Exact solution at h = 1')
#ax2.plot(range(2,num_qubits+1),list(map(lambda x: corr_x_th(num_qubits,x),range(1,num_qubits))),'r--',
         #label = 'Exact solution at h = 1')
ax2.axhline(y = 0, color = 'k', ls = '--', lw = 0.8)
for i in range(len(h_values)):
    ax1.plot(range(2,num_qubits+1),corr_x_abc[i][:], ls = ':', marker = "o", color = colors_blue[i])#, label = f"h = {h_values[i]}")
    ax2.plot(range(2,num_qubits+1),corr_x_pbc[i][:], ls = ':', marker = "o", color = colors_blue[i], label = f"h = {h_values[i]}")

ax1.set_title("ABC")
ax1.set_ylabel("$\\langle\sigma_{1}^{x}\sigma_{i}^{x}\\rangle$")
ax1.set_xlabel("Spin index")
ax1.set_ylim(-0.05,1.05)
ax1.set_xticks([2*i for i in range(1,8)])
ax1.legend(bbox_to_anchor=(0.7,1.28), loc = 'upper right', frameon = False)
ax2.set_title("PBC")
ax2.set_xlabel("Spin index")
ax2.set_ylim(-0.65,1.05)
ax2.set_xticks([2*i for i in range(1,8)])
ax2.legend(bbox_to_anchor=(0.88,1.28), ncols = 3, loc = 'upper right', frameon = False)
#plt.savefig("Correlation_XX_vs_Spin.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

In [ ]:
num_qubits = 14
J = 1.
h_values = [0.5,0.75,1.,1.25,1.5]
corr_z_abc = np.empty([len(h_values),num_qubits - 1],dtype=float)
corr_z_pbc = np.empty([len(h_values),num_qubits - 1],dtype=float)

for i in range(len(h_values)):
    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                       J = J,
                                       h = h_values[i],
                                       BoundaryConditions = 0);
    _,Ground_State,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits,
                                             Quad_Hamiltonian = IsingH,
                                             repetitions = 300000);
    magnetization_z = ExpectedMagnetization(num_qubits = num_qubits, results = HistogramResults)
    corr_h = Correlation(num_qubits, IsingH, "Z", "Z", repetitions = 300000)
    for j in range(num_qubits-1):
        corr_z_abc[i][j] = corr_h[j] - magnetization_z*magnetization_z

    _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                        J = J,
                                        h = h_values[i],
                                        BoundaryConditions = 1);
    _,Ground_State,_,HistogramResults = SimulateHamiltonian(num_qubits = num_qubits,
                                            Quad_Hamiltonian = IsingH,
                                            repetitions = 300000);
    magnetization_z = ExpectedMagnetization(num_qubits = num_qubits, results = HistogramResults)
    corr_h = Correlation(num_qubits, IsingH, "Z", "Z", repetitions = 300000)
    for j in range(num_qubits-1):
        corr_z_pbc[i][j] = corr_h[j] - magnetization_z*magnetization_z

In [ ]:
def corr_z_th(num_qubits, n):
    if n <= num_qubits/2:
        corr = (2/np.pi)*(2/np.pi)*(1/(4*n*n - 1))
    else:
        corr = (2/np.pi)*(2/np.pi)*(1/(4*(num_qubits - n)*(num_qubits - n) - 1))
    return corr

fig, (ax1,ax2) = plt.subplots(1,2,figsize = (8,4))

colors_blue = plt.colormaps['winter'](np.linspace(0,1,len(h_values)))

ax1.plot(range(2,num_qubits+1),list(map(lambda x: corr_z_th(num_qubits,x),range(1,num_qubits))),'r-',label = 'Exact Solution at h = 1')
#ax2.plot(range(2,num_qubits+1),list(map(lambda x: corr_z_th(num_qubits,x),range(1,num_qubits))),'r--',
        #label = 'Exact Solution at h = 1')#,label = 'Theoretical\nat c.p.')

for i in range(len(h_values)):
    ax1.plot(range(2,num_qubits+1),corr_z_abc[i][:],ls = ':',marker = "o", color = colors_blue[i])#,label = f"h = {h_values[i]}")
    ax2.plot(range(2,num_qubits+1),corr_z_pbc[i][:],ls = ':',marker = "o", color = colors_blue[i], label = f"h = {h_values[i]}")

ax1.set_title("ABC")
ax1.set_ylabel("$\\langle\sigma_{1}^{z}\sigma_{i}^{z}\\rangle$ - $\\langle\sigma^{z}\\rangle^{2}$")
ax1.set_xlabel("Spin index")
ax1.set_xticks([2*i for i in range(1,8)])
ax1.set_ylim(-0.01,0.17)
ax1.legend(bbox_to_anchor=(0.7,1.28), loc = 'upper right', frameon = False)
ax2.set_title("PBC")
ax2.set_xticks([2*i for i in range(1,8)])
ax2.set_ylim(-0.01,0.17)
ax2.set_xlabel("Spin index")
ax2.legend(bbox_to_anchor=(0.88,1.28), ncols = 3, loc = 'upper right', frameon = False)
plt.savefig("Correlation_ZZ_vs_Spin.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Bipartite entanglement entropy

In [ ]:
num_qubits = [2,4,6,8,10,12,14]
J = 1.

h_values = np.linspace(0,2,81)
S_abc = np.empty([len(h_values),2,len(num_qubits)],dtype=float)
S_pbc = np.empty([len(h_values),2,len(num_qubits)],dtype=float)
S_obc = np.empty([len(h_values),2,len(num_qubits)],dtype=float)

for l in range(len(num_qubits)):
    for i in range(len(h_values)):
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          periodic = True,
                                          BoundaryConditions = 1)
        _,Ground_State,_,_ = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                Quad_Hamiltonian = IsingH,
                                                repetitions = 1)
        Ground_State_1 = cirq.partial_trace_of_state_vector_as_mixture(Ground_State,list(range(num_qubits[l]//2)))
        Ground_State_2 = cirq.partial_trace_of_state_vector_as_mixture(Ground_State,list(range(num_qubits[l]//2,num_qubits[l])))
    
        Ground_State_fh = np.zeros([2**(num_qubits[l]//2),2**(num_qubits[l]//2)],dtype = complex)#first half
        Ground_State_lh = np.zeros([2**(num_qubits[l]//2),2**(num_qubits[l]//2)],dtype = complex)#last half
        for coef,state in Ground_State_1:
            Ground_State_fh += coef*cirq.density_matrix_from_state_vector(state)
        for coef,state in Ground_State_2:
            Ground_State_lh += coef*cirq.density_matrix_from_state_vector(state)
    
        S_pbc[i,0,l] = np.real(cirq.von_neumann_entropy(Ground_State_fh, atol=1e-5))
        S_pbc[i,1,l] = np.real(cirq.von_neumann_entropy(Ground_State_lh, atol=1e-5))
        #---------------
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          periodic = True,
                                          BoundaryConditions = 0)
        _,Ground_State,_,_ = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                Quad_Hamiltonian = IsingH,
                                                repetitions = 1)
        Ground_State_1 = cirq.partial_trace_of_state_vector_as_mixture(Ground_State,list(range(num_qubits[l]//2)))
        Ground_State_2 = cirq.partial_trace_of_state_vector_as_mixture(Ground_State,list(range(num_qubits[l]//2,num_qubits[l])))
    
        Ground_State_fh = np.zeros([2**(num_qubits[l]//2),2**(num_qubits[l]//2)],dtype = complex)#first half
        Ground_State_lh = np.zeros([2**(num_qubits[l]//2),2**(num_qubits[l]//2)],dtype = complex)#last half
        for coef,state in Ground_State_1:
            Ground_State_fh += coef*cirq.density_matrix_from_state_vector(state)
        for coef,state in Ground_State_2:
            Ground_State_lh += coef*cirq.density_matrix_from_state_vector(state)
    
        S_abc[i,0,l] = np.real(cirq.von_neumann_entropy(Ground_State_fh, atol=1e-5))
        S_abc[i,1,l] = np.real(cirq.von_neumann_entropy(Ground_State_lh, atol=1e-5))
        #--------------
        _,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits[l],
                                          J = J,
                                          h = h_values[i],
                                          periodic = False)
        _,Ground_State,_,_ = SimulateHamiltonian(num_qubits = num_qubits[l],
                                                Quad_Hamiltonian = IsingH,
                                                repetitions = 1)
        Ground_State_1 = cirq.partial_trace_of_state_vector_as_mixture(Ground_State,list(range(num_qubits[l]//2)))
        Ground_State_2 = cirq.partial_trace_of_state_vector_as_mixture(Ground_State,list(range(num_qubits[l]//2,num_qubits[l])))
    
        Ground_State_fh = np.zeros([2**(num_qubits[l]//2),2**(num_qubits[l]//2)],dtype = complex)#first half
        Ground_State_lh = np.zeros([2**(num_qubits[l]//2),2**(num_qubits[l]//2)],dtype = complex)#last half
        for coef,state in Ground_State_1:
            Ground_State_fh += coef*cirq.density_matrix_from_state_vector(state)
        for coef,state in Ground_State_2:
            Ground_State_lh += coef*cirq.density_matrix_from_state_vector(state)
    
        S_obc[i,0,l] = np.real(cirq.von_neumann_entropy(Ground_State_fh, atol=1e-5))
        S_obc[i,1,l] = np.real(cirq.von_neumann_entropy(Ground_State_lh, atol=1e-5))

In [ ]:
fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6)) = plt.subplots(3,2,figsize=(8,8))

colors_blue = plt.colormaps['winter'](np.linspace(0,1,len(num_qubits)))
colors_red = plt.colormaps['autumn'](np.linspace(0,1,len(num_qubits)))
colors_jet = plt.colormaps['jet'](np.linspace(0,1,len(num_qubits)))

for l in range(len(num_qubits)):
    ax1.plot(h_values,S_abc[:,0,l], color = colors_jet[l], label = f' {num_qubits[l]} spins')
    ax2.plot(h_values,S_abc[:,1,l], color = colors_jet[l], label = f' {num_qubits[l]} spins')
    ax3.plot(h_values,S_pbc[:,0,l], color = colors_jet[l], label = f' {num_qubits[l]} spins')
    ax4.plot(h_values,S_pbc[:,1,l], color = colors_jet[l], label = f' {num_qubits[l]} spins')
    ax5.plot(h_values,S_obc[:,0,l], color = colors_jet[l], label = f' {num_qubits[l]} spins')
    ax6.plot(h_values,S_obc[:,1,l], color = colors_jet[l], label = f' {num_qubits[l]} spins')
ax1.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
#ax1.set_xlabel("h")
ax1.set_ylabel('S[$\\rho^{ABC}$]')
ax1.set_ylim(-0.1,1.2)
ax1.set_yticks([0.2*i for i in range(7)])
ax1.set_title("First half $\equiv \\rho =  \\rho_{0, \cdots, L/2 - 1}$")
ax1.legend(bbox_to_anchor=(1.94,1.5), loc = 'upper right', ncols = 4, frameon = False)

ax2.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
#ax2.set_xlabel("h")
#ax2.set_ylabel('S[$\\rho^{ABC}$]')
ax2.set_ylim(-0.1,1.2)
ax2.set_yticks([0.2*i for i in range(7)])
ax2.set_title("Second half $\equiv \\rho = \\rho_{L/2, \cdots, L-1}$")
#ax2.legend(bbox_to_anchor=(1.,1.5), loc = 'upper right', ncols = 3, frameon = False)

ax3.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
#ax3.set_xlabel("h")
ax3.set_ylabel('S[$\\rho^{PBC}$]')
ax3.set_ylim(-0.1,1.2)
ax3.set_yticks([0.2*i for i in range(7)])
#ax3.legend(bbox_to_anchor=(1.,1.7), loc = 'upper right',ncols = 3)

ax4.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
#ax4.set_xlabel("h")
ax4.set_ylim(-0.1,1.2)
ax4.set_yticks([0.2*i for i in range(7)])
#ax4.set_ylabel('S[$\\rho^{PBC}$]')

ax5.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax5.set_xlabel("h")
ax5.set_ylim(-0.1,1.2)
ax5.set_yticks([0.2*i for i in range(7)])
ax5.set_ylabel('S[$\\rho^{OBC}$]')
#ax5.legend(bbox_to_anchor=(1.,1.7), loc = 'upper right',ncols = 3)

ax6.axvline(x = 1, color = 'k', ls = '--', lw = 0.8)
ax6.set_xlabel("h")
ax6.set_ylim(-0.1,1.2)
ax6.set_yticks([0.2*i for i in range(7)])
#ax6.set_ylabel('S[$\\rho^{OBC}$]')
#ax4.legend(bbox_to_anchor=(.95,1.5), loc = 'upper right',ncols = 2)
#plt.savefig("Bipartite_Entanglement_Entropy_2_2_14.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=(8,8))

index = [0,20,40,60,80]

colors_blue = plt.colormaps['winter'](np.linspace(0,1,len(index)))
colors_red = plt.colormaps['autumn'](np.linspace(0,1,len(index)))
colors_green = plt.colormaps['summer'](np.linspace(0,1,len(index)))
colors_jet = plt.colormaps['jet'](np.linspace(0,1,len(index)))

for i,j in enumerate(index):
    ax1.plot(num_qubits,S_abc[j,0,:],marker = 'o',ls = '--', color = colors_jet[i], label = f'h = {np.round(h_values[j],2)}')
    ax2.plot(num_qubits,S_pbc[j,0,:],marker = 'o',ls = '--', color = colors_jet[i], label = f'h = {np.round(h_values[j],2)}')
    ax3.plot(num_qubits,S_obc[j,0,:],marker = 'o',ls = '--', color = colors_jet[i], label = f'h = {np.round(h_values[j],2)}')

ax3.set_xlabel('Number of spins')
ax1.set_title('First half $\equiv \\rho =  \\rho_{0, \cdots, L/2 - 1}$')
ax1.set_ylabel('S[$\\rho^{ABC}$]')
ax2.set_ylabel('S[$\\rho^{PBC}$]')
ax3.set_ylabel('S[$\\rho^{OBC}$]')
ax1.legend(bbox_to_anchor=(0.95,1.4), loc = 'upper right', ncols = 5, frameon = False)
#ax2.legend(bbox_to_anchor=(1.3,0.5), loc = 'right', frameon = False)
#ax3.legend(bbox_to_anchor=(1.3,0.5), loc = 'right', frameon = False)
ax1.set_ylim(-0.1,1.1)
ax2.set_ylim(-0.1,1.1)
ax3.set_ylim(-0.1,1.1)

plt.savefig("Bipartite_Entanglement_Entropy.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Quick data fit and analysis

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def entropy(x,a,c):
    y = a*np.log2(x) + c
    return y

In [ ]:
coef_pbc, cov_pbc = curve_fit(entropy, [4,8,10,12,14], S_pbc[40,0,[False,True,False,True,True,True,True]],p0=[0.16,1.])
coef_abc, cov_abc = curve_fit(entropy, num_qubits, S_abc[40,0,:])
coef_obc, cov_obc = curve_fit(entropy, num_qubits, S_obc[40,0,:],p0=[0.16,1.])

In [ ]:
print("PBC")
for i in range(2):
    print(coef_pbc[i], cov_pbc[i,i],sep='\t')
print("ABC")
for i in range(2):
    print(coef_abc[i], cov_abc[i,i],sep='\t')
print("OBC")
for i in range(2):
    print(coef_obc[i], cov_obc[i,i],sep='\t')

In [ ]:
fig, ax = plt.subplots(figsize = (8,4))

ax.plot(num_qubits,list(map(lambda x: entropy(x,*coef_pbc),num_qubits)), ls = '-', color = 'turquoise', label = 'Fitted curve (PBC)')
ax.plot(num_qubits,list(map(lambda x: entropy(x,*coef_abc),num_qubits)), ls = '-', color = 'orange', label = 'Fitted curve (ABC)')
ax.plot(num_qubits,list(map(lambda x: entropy(x,*coef_obc),num_qubits)), ls = '-', color = 'yellowgreen', label = 'Fitted curve (OBC)')
ax.plot(num_qubits,S_pbc[40,0,:],marker = 'o',ls = 'none', color = 'b', label = 'Calculated (PBC)')
ax.plot(num_qubits,S_abc[40,0,:],marker = 'o',ls = 'none', color = 'r', label = 'Calculated (ABC)')
ax.plot(num_qubits,S_obc[40,0,:],marker = 'o',ls = 'none', color = 'g', label = 'Calculated (ABC)')
ax.set_ylabel('S[$\\rho_{0, \cdots, L/2 - 1}$]')
ax.set_xlabel('Number of spins')
ax.text(11, 0.3, 'Fitting function :\n$y(x) = alog_{2}(x) + b,$\nwith parameters a and b.', fontsize=10)
ax.legend(loc = 'lower right', ncols = 2, frameon = False)
plt.savefig("Bipartite_Entanglement_Entropy_Fitting.svg", dpi = 'figure', format = 'svg', bbox_inches = 'tight')
plt.show()

## Lindblad Operators definitions

Define an Ising Hamiltonian

In [ ]:
#Set parameters
num_qubits = 4 #number of qubits to simulate
J = 1. #interaction 
h = 0.5 #transverse fiel intensitiy
BoundaryConditions = 1 #for PBC 

In [ ]:
FermionIsing,IsingH = QuantumIsingHamiltonian(num_qubits = num_qubits,
                                  J = J,
                                  h = h,
                                  periodic = True,
                                  BoundaryConditions = BoundaryConditions,
                                  printModel=True);
_, Ground_State, _, _ = SimulateHamiltonian(num_qubits = num_qubits,
                                            Quad_Hamiltonian = IsingH,
                                            repetitions=1)#,InitialState=[0,1]);
GetEnergies(Quad_Hamiltonian = IsingH,Final_st_vector=Ground_State,printEnergies=True);

In [ ]:
cirq.dirac_notation(Ground_State)

In [ ]:
sparse_IsingH = get_sparse_operator(IsingH, n_qubits = num_qubits)
np.savez_compressed(f'H_spins_{num_qubits}_J_{J}_h_{h}_BC_{BoundaryConditions}',
                    sparse_IsingH.toarray())
np.savez_compressed(f'G_st_spins_{num_qubits}_J_{J}_h_{h}_BC_{BoundaryConditions}',Ground_State)

### $\sigma^{-}$ operator

We define the $\sigma^{-}$ operator as:
\begin{equation}
    \sigma^{-}=\frac{\sigma^{x}-i\sigma^{y}}{2}=
    \begin{pmatrix}
        0 & 0 \\ 
        1 & 0 
    \end{pmatrix}
\end{equation}

In [ ]:
#set position where sigma minus operator acts
position = 3

#define operator
sm_unique_position = QubitOperator(f"X{position}",1/2) - QubitOperator(f"Y{position}",1j/2)

#save operator
np.savez_compressed(f"sm_op_spins_{num_qubits}_pos_{position}",
    get_sparse_operator(sm_unique_position,n_qubits = num_qubits).toarray())

In [ ]:
sm_all_positions = QubitOperator('')
for i in range(num_qubits):
    #define operator in each i-th position
    sm_all_positions *= (QubitOperator(f"X{i}",1/2) - QubitOperator(f"Y{i}",1j/2))

#save operator
np.savez_compressed(f"sm_op_spins_{num_qubits}_pos_all",
    get_sparse_operator(sm_all_positions,n_qubits = num_qubits).toarray())

### $\sigma^{+}$ operator

We define the $\sigma^{+}$ operator as:
\begin{equation}
    \sigma^{+}=\frac{\sigma^{x}+i\sigma^{y}}{2}=
    \begin{pmatrix}
        0 & 1 \\ 
        0 & 0 
    \end{pmatrix}
\end{equation}

In [ ]:
#set position where sigma minus operator acts
position = 3

#define operator
sp_unique_position = QubitOperator(f"X{position}",1/2) + QubitOperator(f"Y{position}",1j/2)

#save operator
np.savez_compressed(f"sp_op_spins_{num_qubits}_pos_{position}",
    get_sparse_operator(sp_unique_position,n_qubits = num_qubits).toarray())

In [ ]:
sp_all_positions = QubitOperator('')
for i in range(num_qubits):
    #define operator in each i-th position
    sp_all_positions *= (QubitOperator(f"X{i}",1/2) + QubitOperator(f"Y{i}",1j/2))

#save operator
np.savez_compressed(f"sp_op_spins_{num_qubits}_pos_all",
    get_sparse_operator(sp_all_positions,n_qubits = num_qubits).toarray())